# Выбор локации для нефтяной скважины

Необходимо решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats as st

In [2]:
# Прочитаем исходные файлы
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

# Изучим полученные таблицы
datas = [geo_data_0, geo_data_1, geo_data_2]
for file in datas:
    display(file.info())
    display(file.shape)
    display(file.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

(100000, 5)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


...

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

(100000, 5)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


**Вывод**

Мы видим, что каждая таблица содержит по 100 тыс. ненулевых записей, имеет по 5 колонок. Формат данных в графе id - object, а в остальных графах - float64. Данные в графе id, скорее всего помешают при обучении модели и не имеют для нас никакого значения.

## Обучение и проверка модели

In [3]:
# Удалим не нужный столбец
geo_data_0 = geo_data_0.drop(['id'] , axis=1)
    
# Выделим целевые признаки и условия
target = geo_data_0['product']
features = geo_data_0.drop(['product'] , axis=1)
    
# Разобьём данные на выборки
features_train, features_valid, target_train_0, target_valid_0 = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
print("Регион 0")
# Проверим, как распеределились данные
print(len(features_train) / (len(features_train) + len(features_valid)), len(target_train_0) / 
      (len(target_train_0) + len(target_valid_0)))

# Обучим модель, сделаем предсказание, посчитаем rmse, среднее и суммарное значения по предсказанным данным
model = LinearRegression()
model.fit(features_train, target_train_0)
predicted_valid_0 = model.predict(features_valid)
mean_predict = predicted_valid_0.mean()
rmse = (mean_squared_error(target_valid_0, predicted_valid_0)) ** 0.5
        
print('RMSE предсказанного объема сырья:', rmse)
print('Среднее значение предсказанного объема сырья:', mean_predict)
print()

# Удалим не нужный столбец
geo_data_1 = geo_data_1.drop(['id'] , axis=1)
    
# Выделим целевые признаки и условия
target = geo_data_1['product']
features = geo_data_1.drop(['product'] , axis=1)
    
# Разобьём данные на выборки
features_train, features_valid, target_train_1, target_valid_1 = train_test_split(
    features, target, test_size=0.25, random_state=12345)

print("Регион 1")
# Проверим, как распеределились данные
print(len(features_train) / (len(features_train) + len(features_valid)), len(target_train_1) / 
      (len(target_train_1) + len(target_valid_1)))

# Обучим модель, сделаем предсказание, посчитаем rmse, среднее и суммарное значения по предсказанным данным
model = LinearRegression()
model.fit(features_train, target_train_1)
predicted_valid_1 = model.predict(features_valid)
mean_predict = predicted_valid_1.mean()
rmse = (mean_squared_error(target_valid_1, predicted_valid_1)) ** 0.5
        
print('RMSE предсказанного объема сырья:', rmse)
print('Среднее значение предсказанного объема сырья:', mean_predict)
print()

# Удалим не нужный столбец
geo_data_2 = geo_data_2.drop(['id'] , axis=1)
    
# Выделим целевые признаки и условия
target = geo_data_2['product']
features = geo_data_2.drop(['product'] , axis=1)
    
# Разобьём данные на выборки
features_train, features_valid, target_train_2, target_valid_2 = train_test_split(
    features, target, test_size=0.25, random_state=12345)

print("Регион 2")    
# Проверим, как распеределились данные
print(len(features_train) / (len(features_train) + len(features_valid)), len(target_train_2) / 
      (len(target_train_2) + len(target_valid_2)))

# Обучим модель, сделаем предсказание, посчитаем rmse, среднее и суммарное значения по предсказанным данным
model = LinearRegression()
model.fit(features_train, target_train_2)
predicted_valid_2 = model.predict(features_valid)
mean_predict = predicted_valid_2.mean()
rmse = (mean_squared_error(target_valid_2, predicted_valid_2)) ** 0.5
        
print('RMSE предсказанного объема сырья:', rmse)
print('Среднее значение предсказанного объема сырья:', mean_predict)

Регион 0
0.75 0.75
RMSE предсказанного объема сырья: 37.5794217150813
Среднее значение предсказанного объема сырья: 92.59256778438038

Регион 1
0.75 0.75
RMSE предсказанного объема сырья: 0.893099286775616
Среднее значение предсказанного объема сырья: 68.728546895446

Регион 2
0.75 0.75
RMSE предсказанного объема сырья: 40.02970873393434
Среднее значение предсказанного объема сырья: 94.96504596800489


**Вывод**

Проанализировав полученные данные мы видим, что ситуация по регионам существенно различается. В регионе 1 разброс данных незначительный по сравнению с регионами 0 и 2. При этом среднее значение предсказанного объема сырья для региона 1 значительно ниже, чем для остальных регионов.

## Подготовка к расчёту прибыли

In [4]:
# Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. 
one_point_cost = 10000000000 / 200
value_min = one_point_cost / 450000
print('Достаточный объём сырья для безубыточной разработки новой скважины: {:.1f} тыс. барр.'.format(value_min))

# Рассчитаем средний запас сырья в каждом регионе.
product_mean_0 = geo_data_0['product'].mean()
product_mean_1 = geo_data_1['product'].mean()
product_mean_2 = geo_data_2['product'].mean()
print('Средние запасы в регионе 0: {:.1f} тыс. барр.'.format(product_mean_0))
print('Средние запасы в регионе 1: {:.1f} тыс. барр.'.format(product_mean_1))
print('Средние запасы в регионе 2: {:.1f} тыс. барр.'.format(product_mean_2))

# Сравним полученный достаточный объём сырья для безубыточной разработки со средним запасом в каждом регионе.
print(product_mean_0 / value_min)
print(product_mean_1 / value_min)
print(product_mean_2 / value_min)

# Напишем функцию для расчета прибыли
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 450000 * selected.sum() - 10000000000

Достаточный объём сырья для безубыточной разработки новой скважины: 111.1 тыс. барр.
Средние запасы в регионе 0: 92.5 тыс. барр.
Средние запасы в регионе 1: 68.8 тыс. барр.
Средние запасы в регионе 2: 95.0 тыс. барр.
0.8325000000000001
0.6194250000000001
0.8550000000000003


**Вывод**

Мы видим, что средний запас сырья во всех регионах ниже достаточного объём сырья для безубыточной разработки новой скважины. Худший результат в регионе 1: 62% от уровня достаточности.

## Расчёт прибыли и рисков 

In [5]:
# Сбросим индексы в целевых таблицах и приведем формат данных списка предсказаний в pd.Series
target_valid_0 = target_valid_0.reset_index(drop=True)
predicted_valid_0 = pd.Series(predicted_valid_0)

target_valid_1 = target_valid_1.reset_index(drop=True)
predicted_valid_1 = pd.Series(predicted_valid_1)

target_valid_2 = target_valid_2.reset_index(drop=True)
predicted_valid_2 = pd.Series(predicted_valid_2)

state = np.random.RandomState(42) 

# Расчитаем для каждого региона техникой Bootstrap среднюю прибыль, вероятность убытков, 95%-ый доверительный интервал
values = []
for i in range(1000):
    target_subsample = target_valid_0.sample(500, replace=True, random_state=state)
    probs_subsample = predicted_valid_0[target_subsample.index]
    values.append(revenue(target_subsample, probs_subsample, 200))

values = pd.Series(values)
mean = values.mean()
confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())

print("Регион 0")
print("Средняя прибыль: {:.1f} млн. руб.".format(mean / 1000000))
print("Вероятность убытков: {:.1%}".format(len(values.loc[values < 0]) / len(values)))
print("95%-ый доверительный интервал:", confidence_interval)
print()

values = []
for i in range(1000):
    target_subsample = target_valid_1.sample(500, replace=True, random_state=state)
    probs_subsample = predicted_valid_1[target_subsample.index]
    values.append(revenue(target_subsample, probs_subsample, 200))
 
values = pd.Series(values)
mean = values.mean()
confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())

print("Регион 1")
print("Средняя прибыль: {:.1f} млн. руб.".format(mean / 1000000))
print("Вероятность убытков: {:.1%}".format(len(values.loc[values < 0]) / len(values)))
print("95%-ый доверительный интервал:", confidence_interval)
print()

values = []
for i in range(1000):
    target_subsample = target_valid_2.sample(500, replace=True, random_state=state)
    probs_subsample = predicted_valid_2[target_subsample.index]
    values.append(revenue(target_subsample, probs_subsample, 200))
 
values = pd.Series(values)
mean = values.mean()
confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())

print("Регион 2")
print("Средняя прибыль: {:.1f} млн. руб.".format(mean / 1000000))
print("Вероятность убытков: {:.1%}".format(len(values.loc[values < 0]) / len(values)))
print("95%-ый доверительный интервал:", confidence_interval)

Регион 0
Средняя прибыль: 408.8 млн. руб.
Вероятность убытков: 6.6%
95%-ый доверительный интервал: (392159649.28999686, 425520962.8677161)

Регион 1
Средняя прибыль: 513.3 млн. руб.
Вероятность убытков: 1.6%
95%-ый доверительный интервал: (499270889.55298537, 527247359.8685221)

Регион 2
Средняя прибыль: 439.4 млн. руб.
Вероятность убытков: 5.3%
95%-ый доверительный интервал: (422258996.4553227, 456502767.2918152)


**Вывод**

Для разработки скважин предлагается выбрать регион 2. В данном регионе наименьший из трех регионов средний запас сырья. Однако цель любого бизнеса - максимизация прибыли. В регионе 2 мы можем получить максимальную прибыль, а риск убытков минимален.